In [164]:
import sympy as sp
import numpy as np
import pandas as pd
from pprint import pprint as pp
from scipy.sparse import csc_matrix

In [165]:
class NRMethod:
    
    def __init__(self):
        # self.bus_names_indices = {'Allen':1,
        #                           'Betty':2,
        #                           'Clyde':3,
        #                           'Doug':4,
        #                           'Eve':5
        #                           }
        
        # self.z = {'Allen-Betty':0.009+0.0041j,
        #           'Allen-Doug':00.007+0.055j,
        #           'Doug-Clyde':0.011+0.061j,
        #           'Doug-Eve':0.006+0.045j,
        #           'Eve-Clyde':0.010+0.051j
        #           }
        
        self.data = pd.DataFrame({
            'Bus': ['Allen', 'Betty', 'Clyde', 'Doug', 'Eve'],
            'Type': ['slack', 'pv', 'pq', 'pq', 'pq'],
            'index': [1,2,3,4,5],
            'p_gen': [0,210,0,0,0],
            'q_gen': [0,50,0,0,0],
            'p_load': [0,0,110,100,150],
            'q_load': [0,0,85,95,120],
            'q_cap': [0,0,150,50,0],
            'volts': [0.98,1,1,1,1]
            })
        
        self.tl_details = pd.DataFrame({
            'From': ['Allen-1', 'Betty-2','Allen-1', 'Doug-4', 'Doug-4', 'Clyde-3'],
            'To': ['Betty-2', 'Eve-5', 'Doug-4', 'Eve-5', 'Clyde-3', 'Eve-5'],
            'Rse': [0.009,0.006,0.007,0.006,0.011,0.010],
            'Xse': [0.041,0.037,0.055,0.045,0.061,0.051],
            'MVA': [125,250, 200,125,80,75]
            })
        
        
        # flat start
        self.delta2 = 0
        self.delta3 = 0
        self.delta4 = 0
        self.delta5 = 0
        self.v3 = 1
        self.v4 = 1
        self.v5 = 1

In [166]:
# This is just a decorator to attach methods to the class. Annoying jyputer stuff

def attach_to (cls):
    def wrapper(func):
        setattr(cls, func.__name__, func)
        return func
    return wrapper

In [167]:
@attach_to(NRMethod)
def round_complex(self, num):
        '''
        Not called in main.
        '''
        real = round(num.real, 3)
        imag = round(num.imag, 3)
        return complex(real, imag)

In [168]:
@attach_to(NRMethod)
def rect_polar(self, num):
        '''
        Not called in main.
        '''
        mag = round(abs(num), 3)
        ang = round(np.angle(num, deg=True), 3)
        return mag, ang

In [ ]:
@attach_to(NRMethod)
def y_bus (self):
        '''
        Called in main. 
            - Create the Y bus matrix. A fundamental part of the NR solution
            - Create the polar and rectangular forms of the Y values. Makes it easier to process later.
        '''
        y = {'Y_complex':[], 'Y_magnitude':[], 'Y_angle':[]}
        for index, row in self.tl_details.iterrows():
            r = row['Rse']
            x = row['Xse']
            z = complex(r,x)
            y_comp = self.round_complex(num=1/z)
            y_mag, y_ang = self.rect_polar(num=y_comp)
            y['Y_complex'].append(y_comp)
            y['Y_magnitude'].append(y_mag)
            y['Y_angle'].append(y_ang)
        # concatenate self.y into self.tl_details
        self.tl_details_conc = pd.concat([self.tl_details, pd.DataFrame(y)], axis=1)


In [170]:
@attach_to(NRMethod)
def unknown_vector(self):
        '''
        Called in main. Creating the output vector, containing the voltage and the
        angle vectors for each unknown bus.
        
        
        '''
        
        unknown = sp.symbols(f"P1 theta2 theta3 theta4 theta5")
        unknown_vec = sp.Matrix(unknown)
        return unknown_vec

In [171]:
@attach_to(NRMethod)
def known_vector(self):
    known = sp.symbols("delta2 delta3 delta4 delta5 V3 V4 V5")
    known_vec = sp.Matrix(known)
    return known_vec

In [172]:
nr = NRMethod()

In [173]:
nr.y_bus()
nr.unknown_vector()

Matrix([
[    P1],
[theta2],
[theta3],
[theta4],
[theta5]])

In [174]:
nr.known_vector()
y_bus_matrix = []
y_diag = []
y_offdiag = []
for key, val in nr.tl_details_conc.iterrows():
    if val['From'] == val['To']:
        y_diag.append(f"{val['Y_complex']}")
    else:
        y_offdiag.append(f"{val['Y_complex']}")
    # y = f"y:{val['From']}-{val['To']}"
    # print(y)
    # y_bus_matrix.append(f"{val['Y_complex']}")
    # print(y_bus_matrix)
# print(y_offdiag)
# print(nr.tl_details_conc)